# Assignment 6.8 Model Building

Nikki Aaron (na5zn), Kevin Hoffman (keh4nb), Ashley Scurlock (ams5zx), Siddharth Surapaneni (sss2ea)

In [1]:
import pandas as pd
import os

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession \
    .builder \
    .master("local[8]") \
    .appName("solar") \
    .config("spark.driver.memory",'100G') \
    .config('spark.default.parallelism', 16)\
    .getOrCreate()
sc = spark.sparkContext
sc.setCheckpointDir('./checkpoints')

In [2]:
from pyspark.ml.regression import LinearRegression

# Load training data
df = spark.read.csv("data_combined.csv", inferSchema =True, header=True).repartition(500)
df.printSchema()

root
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- timezone: string (nullable = true)
 |-- county_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- state_name: string (nullable = true)
 |-- population: double (nullable = true)
 |-- density: double (nullable = true)
 |-- res_rate: double (nullable = true)
 |-- annual_mbtu_used: double (nullable = true)
 |-- annual_kwh_used: double (nullable = true)
 |-- temp_Apr: double (nullable = true)
 |-- temp_Aug: double (nullable = true)
 |-- temp_Dec: double (nullable = true)
 |-- temp_Feb: double (nullable = true)
 |-- temp_Jan: double (nullable = true)
 |-- temp_Jul: double (nullable = true)
 |-- temp_Jun: double (nullable = true)
 |-- temp_Mar: double (nullable = true)
 |-- temp_May: double (nullable = true)
 |-- temp_Nov: double (nullable = true)
 |-- temp_Oct: double (nullable = true)
 |-- temp_Sep: double (nullable 

In [3]:
# Select columns for training
label = "dollars_saved"
feature_to_ignore = ["res_rate", 
                     "zip",
                     "annual_output_w_hrs", 
                     "anuual_output_kwh20_sps", 
                     "percent_current_needs_met", 
                     "country_name", 
                     "county_name",
                     "city", 
                     "state_id", 
                     "state_name", 
                     "timezone"]
#                     "temp_Apr", 
#                     "temp_Aug",
#                     "temp_Dec",
#                     "temp_Feb",
#                     "temp_Jul",
#                     "temp_Jun",
#                     "temp_Mar",
#                     "temp_May",
#                     "temp_Nov",
#                     "temp_Oct",
#                     "temp_Sep"]
features = [c for c in df.columns if c not in feature_to_ignore + [label]]

df_clean = df.select([c for c in df.columns if c not in feature_to_ignore])
df_train, df_test = df_clean.randomSplit([0.8, 0.2], 500)
df_train.take(1)

[Row(lat=30.8494, lng=-86.20229, population=17877.0, density=26.6, annual_mbtu_used=56.2, annual_kwh_used=16470.594134000003, temp_Jan=31.4, pct_cloudy_days_Apr=53.1, pct_cloudy_days_Aug=63.6, pct_cloudy_days_Dec=63.15, pct_cloudy_days_Feb=71.05, pct_cloudy_days_Jan=61.150000000000006, pct_cloudy_days_Jul=66.15, pct_cloudy_days_Jun=57.85, pct_cloudy_days_Mar=61.95, pct_cloudy_days_May=58.55, pct_cloudy_days_Nov=51.7, pct_cloudy_days_Oct=53.400000000000006, pct_cloudy_days_Sep=53.6, dni=5.1744, annual_output_kwh_20_sps=3015.0274242205314, dollars_saved=381.2174559705672)]

## Model 1: Linear Regression
Best hyperparameters: maxIter:100, regParam:0, elasticNetParam: 0

Model Size: 148 KB

RMSE: 65.9148

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StandardScaler
from pyspark.ml import Pipeline 

assembler_features = VectorAssembler(
    inputCols=features,
    outputCol="features")

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

lr = LinearRegression(maxIter=100, labelCol=label, featuresCol="scaled_features", regParam=0.0, elasticNetParam=0.0,)

# Build the pipeline
pipeline = Pipeline(stages=[assembler_features, scaler, lr])

# Fit the pipeline
model = pipeline.fit(df_train)
#model.save("model-regression")
# Make a prediction
prediction = model.transform(df_test)


In [5]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluate the model on training data
prediction.select(["dollars_saved", "prediction"]).show()

evaluator = RegressionEvaluator(
    labelCol=label, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+------------------+------------------+
|     dollars_saved|        prediction|
+------------------+------------------+
| 309.3342204919779|293.61005185519537|
| 382.2387730709182| 363.5218175714407|
| 297.1028470491332| 337.7505596883436|
|389.85693619720223|338.18061100140665|
| 331.1423580819627| 341.3120479147233|
|298.01351691675757|321.80796565206606|
|288.61508982131323|299.90531201410283|
| 380.0042629133563| 409.9592640269442|
| 337.3079471963648| 369.1681453617781|
|302.87126072752625|354.92004332185263|
| 285.6261048310314| 363.8324451333126|
| 450.1469332525246| 349.2711049103759|
|298.41318348642153| 385.9261424771238|
|289.95876586700865|  364.231389301843|
|317.29404297577935|350.49328504293896|
| 353.1318735873656| 351.9040495987658|
| 321.9092395432814|349.04952188103834|
| 579.7005971027172|496.69947483448783|
|291.42452919195284| 288.2683382062047|
| 313.9195139115664| 332.0051818076171|
+------------------+------------------+
only showing top 20 rows

Root Mean Squa

## Model 2: Random Forest Regression
Best hyperparameters: maxDepth: 8, numTrees: 250, minInstancesPerNode: 1

Model Size: 4.9 MB

RMSE: 33.744

In [6]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol=label, maxDepth=8, numTrees=250, minInstancesPerNode=1)

# Chain indexer and forest in a Pipeline
pipeline_rf = Pipeline(stages=[assembler_features, scaler, rf])

model = pipeline_rf.fit(df_train)
#model.save("model-rf")
predictions_rf = model.transform(df_test)

evaluator = RegressionEvaluator(
   labelCol=label, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_rf)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-3fcfcd1f81a0>", line 7, in <module>
    model = pipeline_rf.fit(df_train)
  File "/usr/local/spark/python/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/usr/local/spark/python/pyspark/ml/pipeline.py", line 109, in _fit
    model = stage.fit(dataset)
  File "/usr/local/spark/python/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/usr/local/spark/python/pyspark/ml/wrapper.py", line 295, in _fit
    java_model = self._fit_java(dataset)
  File "/usr/local/spark/python/pyspark/ml/wrapper.py", line 292, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1255, in __call__
    answer = self.gateway_client.send_command(command)
  File

KeyboardInterrupt: 

In [10]:
predictions_rf.select(["dollars_saved", "prediction"]).show()

+-----------------+-----------------+
|    dollars_saved|       prediction|
+-----------------+-----------------+
|484.5031021125742| 516.505343590026|
|484.6682391251149|517.4366364473047|
|516.8557923471669|518.5480942673072|
|516.0491487817686|518.4635486310632|
|516.0173890931666|518.0718040583994|
|515.7661149761013|516.8476291907117|
|515.8825341130485|516.8290975153025|
| 515.864331864375|518.1099441179163|
|516.4802310303666|517.9523654638882|
|515.7580085695923|518.1099441179163|
|484.4272456962049| 516.254079042995|
|516.0233877856182| 518.061149261007|
|516.4444948751209|518.2245319764656|
|516.3146217883103|518.1129351300027|
|516.8782180531416|517.0463894341927|
|516.9301723033138|513.7632972381936|
|516.8058037183525|513.6393711245371|
|516.5456508321224|519.4899607344098|
|516.3220107680502|519.1714144267758|
|516.4141263941766|519.1675502918422|
+-----------------+-----------------+
only showing top 20 rows



## Model 3: Gradient Boosted Trees
Best hyperparameters: maxDepth:8, minInstancesPerNode:2, maxIter: 100, stepSize: 0.1

Model Size: 1.7MB

RMSE: 20.7213

With no temp  maxDepth:8, minInstancesPerNode:4, maxIter: 200, stepSize: 0.1 : 20.029


In [ ]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

gbt = GBTRegressor(labelCol=label, predictionCol='prediction', featuresCol='scaled_features', maxDepth=8, minInstancesPerNode=1, maxIter=150, stepSize=0.1)

paramGrid = ParamGridBuilder()\
  .addGrid(gbt.minInstancesPerNode, [1, 2, 4])\
  .build()

# We define an evaluation metric.  This tells CrossValidator how well we are doing by comparing the true labels with predictions.
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
# Declare the CrossValidator, which runs model tuning for us.
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)
# Chain indexer and forest in a Pipeline
#pipeline_gbt = Pipeline(stages=[assembler_features, scaler, gbt])
pipeline_gbt = Pipeline(stages=[assembler_features, scaler, cv])

# Train model.  This also runs the indexer.
model = pipeline_gbt.fit(df_train)
display(model.bestModel)
#model.save("model-gbt")
# Make predictions.
predictions_gbt = model.transform(df_test)


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol=label, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_gbt)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

In [7]:
predictions_gbt.select(["dollars_saved", "prediction"]).show()

+-----------------+------------------+
|    dollars_saved|        prediction|
+-----------------+------------------+
|484.5031021125742| 506.8656149623041|
|484.6682391251149| 517.2272117653333|
|516.8557923471669| 513.6508706464688|
|516.0491487817686| 518.2075095279591|
|516.0173890931666| 516.7011605913955|
|515.7661149761013|510.68275608126334|
|515.8825341130485|511.71881025903707|
| 515.864331864375| 516.6866065674952|
|516.4802310303666| 512.2948439893455|
|515.7580085695923| 517.1193681228697|
|484.4272456962049| 507.3258785651495|
|516.0233877856182| 516.0678735473616|
|516.4444948751209| 515.7743016470491|
|516.3146217883103| 516.7414089283822|
|516.8782180531416| 513.5438512305753|
|516.9301723033138| 513.9171039350417|
|516.8058037183525| 514.1072459065782|
|516.5456508321224| 515.8038068640749|
|516.3220107680502|  520.957962688935|
|516.4141263941766| 520.8632931932017|
+-----------------+------------------+
only showing top 20 rows



In [10]:
# Save notebook as PDF document
!jupyter nbconvert --to pdf `pwd`/*.ipynb

[NbConvertApp] Converting notebook /sfs/qumulo/qhome/keh4nb/ds5559_project/Solar Models.ipynb to pdf
[NbConvertApp] Writing 42283 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 50554 bytes to /sfs/qumulo/qhome/keh4nb/ds5559_project/Solar Models.pdf
[NbConvertApp] Converting notebook /sfs/qumulo/qhome/keh4nb/ds5559_project/Solar Regression-Copy1.ipynb to pdf
[NbConvertApp] Writing 59218 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF succes